# Binning raw Multidimensional Photoemission Spectroscopy (MPES) data and converting it into the NeXus format

This example shows how to generate xarray based h5 files from WSe2 trARPES measurement data as detailed in this [paper](https://www.nature.com/articles/s41597-020-00769-8) and how to generate a file in the standardised [MPES NeXus format](https://manual.nexusformat.org/classes/contributed_definitions/NXmpes.html#nxmpes) from it.
Due to the size of the example file (~6GB) you need at least 20 GB of memory on your computer you're executing this example on. If you just want to have a look on how to convert a pre-binned xarray based h5 file into the NeXus format you may have a look at the simpler [Convert to NeXus example](./E1%20Convert%20to%20NeXus.ipynb), which has lower hardware requirements.

## Download RAW data (trARPES data of WSe2)

Here, we just import `shutil` for extracting the downloaded zip archive and set the main file folder for holding the measurement data.

In [ ]:
import shutil
import os
FDIR = f'{os.getcwd()}/Scan049_1'
ECAL = f'{os.getcwd()}/energycal_2019_01_08'

Since the provided measurement files are rather large (~6GB), they are not directly provided with the example.
You can [download](https://zenodo.org/record/6369728/files/WSe2.zip) it from zenodo. This may take some time. Place the file in the directory of this notebook afterwards. Under Linux, macOS and in a NORTH container you can directly use the cell below to download the file with curl.

In [ ]:
! curl -o WSe2.zip "https://zenodo.org/record/6369728/files/WSe2.zip"

Now we extract the measurement files.

In [ ]:
shutil.unpack_archive('WSe2.zip')

## Binning of measurement data

First we import the necessary packages. For a manual on how to install this dependencies refer to the provided [INSTALL.md](./INSTALL.md) file. If you're running a pre-built docker container or working with the NORTH tools, these dependencies are already available for you.

In [ ]:
from mpes import base as base, fprocessing as fp, analysis as aly
import matplotlib.pyplot as plt
import numpy as np
import os
from dask import compute
import datetime as dt
import h5py

### Initial data binning for distortion correction

In [ ]:
parp = fp.parallelHDF5Processor(folder=FDIR)
parp.gather(identifier=r'/*.h5', file_sorting=True)
len(parp.files)

In [ ]:
parp.files = parp.files[0:50]
axes = ['X', 'Y', 't']
# Important to keep the whole detector area for the initial binning!
bins = [512, 512, 300]
ranges = [(0, 2048), (0, 2048), (64000, 68000)]
parp.parallelBinning(axes=axes, nbins=bins, ranges=ranges, scheduler='threads', ret=False)

### Determine correction landmarks

In [ ]:
mc = aly.MomentumCorrector(parp.combinedresult['binned'])
mc.selectSlice2D(slice(165, 175), 2)
mc.featureExtract(mc.slice, sigma=5, fwhm=10, sigma_radius=3)
mc.view(points=mc.features, annotated=True)

### Calculate thin plate spline symmetry correction

In [ ]:
mc.splineWarpEstimate(image=mc.slice, landmarks=mc.pouter_ord, include_center=True,
                      iterative=False, interp_order=2, update=True)
mc.view(image=mc.slice_transformed, annotated=True, points={'feats':mc.ptargs}, backend='bokeh', crosshair=True, radii=[75,110,150], crosshair_thickness=0.2)

### Image registration

In [ ]:
mc.coordinateTransform(type='translation', xtrans=70., ytrans=70., keep=True)
plt.imshow(mc.slice_transformed, origin='lower', cmap='terrain_r')
plt.axvline(x=256)
plt.axhline(y=256)

In [ ]:
mc.coordinateTransform( type='rotation', angle=-5, center=(256., 256.), keep=True)
plt.imshow(mc.slice_transformed, origin='lower', cmap='terrain_r')
plt.axvline(x=256)
plt.axhline(y=256)

In [ ]:
# Final Deformation field:
subs = 20
plt.scatter(mc.cdeform_field[::subs,::subs].ravel(), mc.rdeform_field[::subs,::subs].ravel(), c='b')

### Momentum calibration

In [ ]:
# Pick one high-symmetry point
point_b = [252.,255.]
# Pick the BZ center
point_a = [308.,346.]
# give the distance of the two in inverse Angstrom
distance = np.pi*4/3/3.297
# Momentum calibration assuming equal scaling along both x and y directions (equiscale=True)
# Requirements : pixel coordinates of and the momentum space distance between two symmetry points, 
# plus the momentum coordinates
# of one of the two points 
ext = mc.calibrate(mc.slice_transformed,
                   point_from=point_a,
                   point_to=point_b,
                   dist=distance,
                   equiscale=True,
                   ret=['extent'])

In [ ]:
mc.view(image=mc.slice_transformed, imkwds=ext)
plt.xlabel('$k_x$', fontsize=15)
plt.ylabel('$k_y$', fontsize=15)

### Energy calibration

In [ ]:
axes = ['t']
bins = [1000]
ranges = [(63000, 80000)]
traces, tof = fp.extractEDC(folder=ECAL,
                            axes=axes, bins=bins, ranges=ranges)

In [ ]:
voltages = np.arange(-12.2, -23.2, -1)
ec = aly.EnergyCalibrator(biases=voltages, traces=traces, tof=tof)

In [ ]:
ec.normalize(smooth=True, span=7, order=1)
ec.view(traces=ec.traces_normed, xaxis=ec.tof, backend='bokeh')

In [ ]:
rg = [(65000, 65200)]
ec.addFeatures(traces=ec.traces_normed, refid=0, ranges=rg[0], infer_others=True, mode='append')
ec.featranges

In [ ]:
ec.featureExtract(traces=ec.traces_normed, ranges=ec.featranges)
ec.view(traces=ec.traces_normed, peaks=ec.peaks, backend='bokeh')

### Calculate energy calibration

In [ ]:
refid=5
Eref=-1.3
axs = ec.calibrate(ret='all', Eref=Eref, t=ec.tof, refid=refid)
ec.view(traces=ec.traces_normed, xaxis=ec.calibration['axis'], backend='bokeh')

### Quality of calibration

In [ ]:
for i in range(0,len(voltages)):
    plt.plot(ec.calibration['axis']-(voltages[i]-voltages[refid]), ec.traces_normed[i])
plt.xlim([-15,5])

### Inspect calibration function

In [ ]:
ec.view(traces=ec.calibration['axis'][None,:], xaxis=ec.tof, backend='matplotlib', show_legend=False)
plt.scatter(ec.peaks[:,0], ec.biases-ec.biases[refid]+Eref, s=50, c='k')
plt.xlabel('Time-of-flight', fontsize=15)
plt.ylabel('Energy (eV)', fontsize=15)
plt.ylim([-8,6])
plt.xlim([63400,69800])

### Dataframe processor

In [ ]:
dfp = fp.dataframeProcessor(datafolder=FDIR)
dfp.read(source='folder', ftype='h5', timeStamps=True)

### Apply energy calibration

In [ ]:
dfp.appendEAxis(E0=ec.calibration['E0'], a=ec.calibration['coeffs'])
dfp.edf.head(8)

### Apply distortion correction

In [ ]:
dfp.applyKCorrection(type='tps_matrix',
                     rdeform_field = mc.rdeform_field,
                     cdeform_field = mc.cdeform_field,
                     X='X', Y='Y', newX='Xm', newY='Ym')
dfp.edf.head(8)

### Apply momentum calibration

In [ ]:
dfp.appendKAxis(point_b[0], point_b[1], X='Xm', Y='Ym', rstart=parp.binranges[0][0],
                cstart=parp.binranges[1][0],
                rstep=parp.binsteps[0],
                cstep=parp.binsteps[1],
                fc=mc.calibration['coeffs'][0],
                fr=mc.calibration['coeffs'][1])
dfp.edf.head(8)

### Apply pump-probe delay axis conversion

In [ ]:
ADCRange = (650, 6900)
timeRange = (-100, 200)
dfp.edf['delay'] = timeRange[0] + (dfp.edf['ADC']-ADCRange[0]) *\
    (timeRange[1] - timeRange[0])/(ADCRange[1]-ADCRange[0])
dfp.edf.head(8)

### Bin 4D data in transformed grid

In [ ]:
axes = ['kx', 'ky', 'E', 'delay']
bins = [50, 50, 100, 21]
ranges = [(-2, 2), (-2, 2), (-3, 2), (-110, 190)]
# jittering of energy and ADC should best be done on the bin size of the hardware, 
# not the rebinned bin size
TOFrange=[64500,67000]
e_t_conversion = (base.tof2evpoly(ec.calibration['coeffs'],
                                  ec.calibration['E0'], 
                                  TOFrange[0])
                  - base.tof2evpoly(ec.calibration['coeffs'],
                                    ec.calibration['E0'], TOFrange[1])
                 ) / (TOFrange[1] - TOFrange[0])
d_adc_conversion = (timeRange[1] - timeRange[0]) / (ADCRange[1] - ADCRange[0])
jitter_amplitude = [0.5,
                    0.5,
                    1*bins[2]/abs(ranges[2][1]-ranges[2][0])*e_t_conversion,
                    1*bins[3]/abs(ranges[3][1]-ranges[3][0])*d_adc_conversion]
dfp.distributedBinning(axes=axes,
                       nbins=bins,
                       ranges=ranges,
                       scheduler='threads',
                       ret=False,
                       jittered=True,
                       jitter_amplitude=jitter_amplitude)

### Create metatada structure
This adds additional metadata to the h5 file. This data may also be provided through additional ELN entries through a NOMAD instance or with a handwriten file directly to the mpes parser.

In [ ]:
metadata = {}

In [ ]:
# manual Meta data
#General
metadata['experiment_summary'] = 'WSe2 XUV NIR pump probe data.'
metadata['entry_title'] = 'Valence Band Dynamics - 800 nm linear s-polarized pump, 0.6 mJ/cm2 absorbed fluence'
metadata['experiment_title'] = 'Valence band dynamics of 2H-WSe2'

#User
# Fill general parameters of NXuser
# TODO: discuss how to deal with multiple users?
metadata['user0'] = {}
metadata['user0']['name'] = 'Julian Maklar'
metadata['user0']['role'] = 'Principal Investigator'
metadata['user0']['affiliation'] = 'Fritz Haber Institute of the Max Planck Society'
metadata['user0']['address'] = 'Faradayweg 4-6, 14195 Berlin'
metadata['user0']['email'] = 'maklar@fhi-berlin.mpg.de'

#NXinstrument
metadata['instrument'] = {}
#analyzer
metadata['instrument']['analyzer']={}
metadata['instrument']['analyzer']['slow_axes'] = "delay" # the scanned axes
metadata['instrument']['analyzer']['spatial_resolution'] = 10.
metadata['instrument']['analyzer']['energy_resolution'] = 110.
metadata['instrument']['analyzer']['momentum_resolution'] = 0.08
metadata['instrument']['analyzer']['projection'] = "reciprocal"
metadata['instrument']['analyzer']['working_distance'] = 4.
metadata['instrument']['analyzer']['lens_mode'] = "6kV_kmodem4.0_30VTOF.sav"

# Need to get those from the data file...
# Put into separate Lens objects?
# metadata['instrument']['analyzer']['fa_size'] = 200
# metadata['instrument']['analyzer']['ca_size'] = np.nan

#probe beam
metadata['instrument']['beam']={}
metadata['instrument']['beam']['probe']={}
metadata['instrument']['beam']['probe']['incident_energy'] = 21.7
metadata['instrument']['beam']['probe']['incident_energy_spread'] = 0.11
metadata['instrument']['beam']['probe']['pulse_duration'] = 20.
metadata['instrument']['beam']['probe']['frequency'] = 500.
metadata['instrument']['beam']['probe']['incident_polarization'] = [1, 1, 0, 0] # p pol Stokes vector
metadata['instrument']['beam']['probe']['extent'] = [80., 80.] 
#pump beam
metadata['instrument']['beam']['pump']={}
metadata['instrument']['beam']['pump']['incident_energy'] = 1.55
metadata['instrument']['beam']['pump']['incident_energy_spread'] = 0.08
metadata['instrument']['beam']['pump']['pulse_duration'] = 35.
metadata['instrument']['beam']['pump']['frequency'] = 500.
metadata['instrument']['beam']['pump']['incident_polarization'] = [1, -1, 0, 0] # s pol Stokes vector
metadata['instrument']['beam']['pump']['incident_wavelength'] = 800. 
metadata['instrument']['beam']['pump']['average_power'] = 300.
metadata['instrument']['beam']['pump']['pulse_energy'] = metadata['instrument']['beam']['pump']['average_power']/metadata['instrument']['beam']['pump']['frequency']#µJ
metadata['instrument']['beam']['pump']['extent'] = [230., 265.] 
metadata['instrument']['beam']['pump']['fluence'] = 0.15

#sample
metadata['sample']={}
metadata['sample']['preparation_date'] = '2019-01-13T10:00:00+00:00'
metadata['sample']['sample_history'] = 'Cleaved'
metadata['sample']['chemical_formula'] = 'WSe2'
metadata['sample']['description'] = 'Sample'
metadata['sample']['name'] = 'WSe2 Single Crystal'
# metadata['sample']['temperature'] = 300.
# metadata['sample']['pressure'] = 5.e-11

### Convert and save as h5 file

In [ ]:
res_xarray = dfp.gather_metadata(metadata_dict=metadata.copy(), ec=ec, mc=mc)

In [ ]:
dfp.xarray_to_h5(res_xarray, "WSe2_xarray.h5")

## Convert the generated file into the NeXus format
This conversion basically follows the same procedure as in the [convert to NeXus example](./E1%20Convert%20to%20Nexus.ipynb). Please refer to this notebook for details on the convert function.

In [ ]:
from nexusparser.tools.dataconverter.convert import convert

In [ ]:
convert(input_file=["WSe2_xarray.h5","config_file.json","ELN_metadata_example.yaml"],
        reader='mpes',
        nxdl='NXmpes',
        output='WSe2.mpes.nxs')

## View the data with H5Web
H5Web is a tool for visualizing any data in the h5 data format. Since the NeXus format builds opon h5 it can be used to view this data as well. We just import the package and call H5Web with the output filename from the convert command above. For an analysis on NeXus data files please refer to [analysis example](./E3%20pyARPES%20analysis.ipynb).

You can also view this data with the H5Viewer or other tools from your local filesystem.

In [ ]:
from jupyterlab_h5web import H5Web

In [ ]:
H5Web('WSe2.mpes.nxs')